In [35]:
# Yolov7 Repository Clone and Requirements Installation:
!git clone https://github.com/WongKinYiu/yolov7.git

In [37]:
# Importing Libraries:
!pip install -U -r requirements.txt

In [38]:
import matplotlib.pyplot as plt
import torch 
import cv2

In [39]:
import math
from torchvision import transforms
import numpy as np

In [40]:
# !pip install telepot
import cv2

In [41]:
import telepot
import os

In [43]:
!pip install utils

In [44]:
# import os
# os.getcwd()


In [54]:
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts


In [55]:
# Telegram Integration:
token = 'xxxxxxxxxxxxxxxxxxx' # telegram token
receiver_id = 000000000 # https://api.telegram.org/bot<TOKEN>/getUpdates
bot = telepot.Bot(token)


In [56]:
# Loading YOLOv7 Model:
import sys
import torch

# Add the directory containing the 'models' module to sys.path
sys.path.append(r'C:\Users\amala\yolov7')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
weights_path = 'yolov7/yolov7-w6-pose.pt'

try:
    # Load the weights
    weights = torch.load(weights_path, map_location=device)
    model = weights['model']

    # Move the model to the device and set to eval mode
    model = model.half().to(device)
    _ = model.eval()

    print("Model loaded successfully.")
except FileNotFoundError:
    print(f"File not found: {weights_path}")
except Exception as e:
    print(f"Error loading model: {e}")


Model loaded successfully.


In [57]:
import telepot

token = '7043918091:AAEbcubnv56WteKGv6Ua1ciRA55KN9_M-90'
receiver_id = 1103585699

bot = telepot.Bot(token)

# Test the connection by sending a test message
try:
    bot.sendMessage(receiver_id, "Test message from your bot")
    print("Test message sent successfully.")
except Exception as e:
    print(f"Error sending test message: {e}")


Error sending test message: HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot7043918091:AAEbcubnv56WteKGv6Ua1ciRA55KN9_M-90/sendMessage (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001F2664C8B90>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))


In [58]:
#Video Processing:
threshold=10
video_path = 'video.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print('Error while trying to read video. Please check the path again.')

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

vid_write_image = letterbox(cap.read()[1], frame_width, stride=64, auto=True)[0]
resize_height, resize_width = vid_write_image.shape[:2]
out_video_name = f"{video_path.split('/')[-1].split('.')[0]}"
# Output Video Generation:
out = cv2.VideoWriter(f"{out_video_name}_keypoint.mp4",
                      cv2.VideoWriter_fourcc(*'mp4v'), 30,
                      (resize_width, resize_height))

In [ ]:
frame_count = 0
fall_detected = False  # Flag to track fall detection
# Convert model weights to float before processing
model.to(torch.float32)

previous_shoulder_height = None
previous_foot_height = None

while cap.isOpened():
    print("Frame {} Processing".format(frame_count))
    frame_count += 1

    ret, frame = cap.read()
    if not ret:
        break

    orig_image = frame
    image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
    image = letterbox(image, frame_width, stride=64, auto=True)[0]
    image = transforms.ToTensor()(image)
    image = torch.tensor(np.array([image.numpy()])).float().to(device)

    with torch.no_grad():
        output, _ = model(image)

    output = output.float()
    output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
    output = output_to_keypoint(output)
    im0 = image[0].permute(1, 2, 0) * 255
    im0 = im0.cpu().numpy().astype(np.uint8)
    im0 = cv2.cvtColor(im0, cv2.COLOR_RGB2BGR)
# Fall Detection Message Handling:
    fall_detected = False  # Reset fall detection flag for each frame

    for idx in range(output.shape[0]):
        xmin, ymin = (output[idx, 2] - output[idx, 4] / 2), (output[idx, 3] - output[idx, 5] / 2)
        xmax, ymax = (output[idx, 2] + output[idx, 4] / 2), (output[idx, 3] + output[idx, 5] / 2)

        # Check for a significant decrease in shoulder and foot heights
        shoulder_height = output[idx][23]
        foot_height = output[idx][53]
        if previous_shoulder_height is not None and previous_foot_height is not None:
            if (previous_shoulder_height - shoulder_height > threshold) and (previous_foot_height - foot_height > threshold):
                fall_detected = True

        print("Shoulder height:", shoulder_height)
        print("Foot height:", foot_height)

        if fall_detected:
            cv2.rectangle(im0, (int(xmin), int(ymin)), (int(xmax), int(ymax)), color=(0, 0, 255), thickness=5, lineType=cv2.LINE_AA)
            cv2.putText(im0, 'Person Fell down', (11, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, [0, 0, 255], thickness=3, lineType=cv2.LINE_AA)
            print("fall_detected value:", fall_detected)

    # Update previous heights
    previous_shoulder_height = shoulder_height
    previous_foot_height = foot_height

    if fall_detected:
        try:
            bot.sendMessage(receiver_id, "Person Fall Detected")
            print("Fall detection message sent successfully.")
        except Exception as e:
            print(f"Error sending fall detection message: {e}")
        fall_detected = False  # Reset the flag after sending the message

    out.write(im0)

cap.release()
out.release()